In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/00.Libs")
import RS_dataset
import RS_models
import RS_utils
import RS_train

In [2]:
import datetime
import logging
import numpy as np 
from glob import glob
import os
import torch
import matplotlib.pyplot as plt 
from torch.utils.data import DataLoader
#---
import torch.nn as nn 
#---
from lightning.fabric import Fabric
import lightning as L
import segmentation_models_pytorch as smp

In [3]:
#-- data
img_path = "/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/01.512_imgs"
mask_path = "/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/04.SAM_fine/0.data/02.512_masks"

img_path_ship  = np.array(sorted(glob(os.path.join(img_path, "*.png"))) )
mask_path_ship = np.array(sorted(glob(os.path.join(mask_path, "*.png"))) )

aa = np.load("/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/05.Training/Segmentation/03.data_list/512_ships.npy")

selected_paths_img = img_path_ship[aa]
selected_paths_mask  = mask_path_ship[aa]

In [4]:
#-- args
TASK = "SHIP"
MODEL_NAME = "UNET_with_EDGE_v"
EXEC_VER = 70 
BATCH_SIZE = 4
DEVICE = "cuda:0"
DEVICES = [0,1,2,3]
RESUME = False
SAVE_EPOCH = 20

In [5]:
#-- category 
ISAID_CLASSES_SHIP = (
    'background','ship','harbor' 
    )
ISAID_PALETTE_SHIP = {
    0: (0, 0, 0), 
    1: (0, 0, 63), 
    2: (0, 100, 155)}

In [6]:
#--- logger
# Set up logging
log_filename = datetime.datetime.now().strftime(f'/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/08.Segmentation_with_Edge/01.log/ver_{EXEC_VER}_%Y-%m-%d_%H-%M-%S.log')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
handler = logging.FileHandler(log_filename)
logger.addHandler(handler)

In [14]:
#-- dataset
train_dataset = RS_dataset.Seg_RS_dataset_edge_v2(img_dir=selected_paths_img, mask_dir=selected_paths_mask, image_resize = None, phase="train",palette=ISAID_PALETTE_SHIP,gaussian=False,mask_onehot=True, softmax=True )
dataloader  = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=train_dataset.collate_fn)

In [15]:
img, label,edge = train_dataset.__getitem__(0)

In [19]:
img.shape

torch.Size([3, 256, 256])

In [18]:
torch.unique(label)

tensor([0.5000, 0.7311])

In [22]:
#--- model 
#model = RS_models.Edge_Net()
model_1 = smp.UnetPlusPlus(encoder_name="resnet152",classes=3)
tgt_path = "/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/07.Unet_PlusPlus_EdgeNet/02.ckpts/ver_35_unet_epoch_101_iteration_470256.pt"
checkpoint = torch.load(tgt_path)
model_1.load_state_dict(checkpoint)

<All keys matched successfully>

In [23]:
model_2 = RS_models.Edge_Net(input_channel=3)
tgt_path = "/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/01.Models/06.Edge_Net/02.ckpts/ver_62_EDGE_EDGE_NET_epoch_81_iteration_6081318.pt"
checkpoint = torch.load(tgt_path)
model_2.load_state_dict(checkpoint)

<All keys matched successfully>

In [25]:
model = RS_models.CombinedModel(model_1, model_2)
model = model.to(DEVICE)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [26]:
epochs = 999
iteration = 0
for epoch in range(epochs):
    epoch_running_loss = 0 

    dice_score = 0
    for index, data in enumerate(dataloader):
        imgs, mask, edge = data
        imgs, mask, edge = imgs.to(DEVICE), mask.to(DEVICE), edge.to(DEVICE)
    
        # opt
        optimizer.zero_grad()
        # runs
        outputs,loss_percept = model(imgs, mask)
        # loss
        loss = outputs + loss_percept
        loss.backward()
        #fabric.backward(loss)
        optimizer.step()

        # stat
        epoch_running_loss += loss.item()

        # log
        logger.info(f"epoch : {epoch} iter : {index} loss: {loss:.5f} dice: {dice_score:.5f} loss_percept: {loss_percept:.5f}" )
        log = {'loss': f'{loss:.5f}' }
        print(log)
        #wandb.log(log)

        iteration += index
        log_iter = 100
        
        if (index % log_iter) == 0:    # print every log_iter mini-batches
            print(f"epoch : {epoch} , total_iter : {iteration} , iter_per_epoch : {len(dataloader)} , running_loss : {epoch_running_loss / (index +1)}")
        
    #-- save     
    if epoch % SAVE_EPOCH ==0:
        #-- save 
        save_path = f"./02.ckpts/ver_{EXEC_VER}_{TASK}_{MODEL_NAME}_epoch_{epoch + 1}_iteration_{iteration}.pt"
        torch.save(model.state_dict(), save_path)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:17                                                                                   │
│                                                                                                  │
│ /mnt/hdd/eric/.conda/envs/8.tmp.copied/lib/python3.8/site-packages/torch/_tensor.py:488 in       │
│ backward                                                                                         │
│                                                                                                  │
│    485 │   │   │   │   create_graph=create_graph,                                                │
│    486 │   │   │   │   inputs=inputs,                                                            │
│    487 │   │   │   )                                                                             │
│ ❱  488 │   │   torch.autograd.backward(                                                          │
│    489 │   │   │   self, gradient, retain_graph, create_graph, inputs=inputs                     │
│    490 │   │   )                                                                                 │
│    491                                                                                           │
│                                                                                                  │
│ /mnt/hdd/eric/.conda/envs/8.tmp.copied/lib/python3.8/site-packages/torch/autograd/__init__.py:19 │
│ 0 in backward                                                                                    │
│                                                                                                  │
│   187 │   │   tuple(inputs) if inputs is not None else tuple()                                   │
│   188 │                                                                                          │
│   189 │   grad_tensors_ = _tensor_or_tensors_to_tuple(grad_tensors, len(tensors))                │
│ ❱ 190 │   grad_tensors_ = _make_grads(tensors, grad_tensors_, is_grads_batched=False)            │
│   191 │   if retain_graph is None:                                                               │
│   192 │   │   retain_graph = create_graph                                                        │
│   193                                                                                            │
│                                                                                                  │
│ /mnt/hdd/eric/.conda/envs/8.tmp.copied/lib/python3.8/site-packages/torch/autograd/__init__.py:85 │
│ in _make_grads                                                                                   │
│                                                                                                  │
│    82 │   │   elif grad is None:                                                                 │
│    83 │   │   │   if out.requires_grad:                                                          │
│    84 │   │   │   │   if out.numel() != 1:                                                       │
│ ❱  85 │   │   │   │   │   raise RuntimeError("grad can be implicitly created only for scalar o   │
│    86 │   │   │   │   new_grads.append(torch.ones_like(out, memory_format=torch.preserve_forma   │
│    87 │   │   │   else:                                                                          │
│    88 │   │   │   │   new_grads.append(None)                                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: grad can be implicitly created only for scalar outputs